### Assignment Objective
This assignment demonstrates a personal chatbot assistant for **Bloom & Vows Floral Wedding**.

The chatbot can interact with clients, answer questions about services, packages, pricing, and contact information, and record leads.

It uses two key documents to understand the business:

**Business Profile (PDF)** that contains all details about Bloom & Vows, including services, packages, pricing, and contact info.

**Business Summary (Text File)** that provides a concise overview of the business and its unique offerings.

The chatbot also uses two tools:

**record_customer_interest** used to save client names, emails, and notes to a CSV.

**record_feedback** used to log any questions the chatbot cannot answer for later review.

With these, the chatbot acts as a knowledgeable digital assistant for the business.

### 1. Install Dependencies

In [23]:
!pip install google-generativeai gradio python-dotenv pypdf pandas

### 2. Import Libraries

In [25]:
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

###3. Load Google API Key

In [56]:
load_dotenv(".env")
google_api_key = os.getenv("OPENAI_API_KEY")

In [50]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

### 4. Define Tools

In [27]:
LEADS_FILE = "leads.csv"
FEEDBACK_FILE = "feedback.csv"


def record_customer_interest(name: str, email: str, message: str) -> str:
    """Logs customer interest to a CSV file."""

    # Default unknown fields if missing
    name = name.strip() if name else "Unknown"
    email = email.strip() if email else "Unknown"
    message = message.strip() if message else "No message provided"

    # Prepare new lead row
    new_lead = {
        "timestamp": datetime.now().isoformat(),
        "name": name,
        "email": email,
        "message": message
    }

    # Append to CSV
    if not os.path.exists(LEADS_FILE):
        df = pd.DataFrame([new_lead])
        df.to_csv(LEADS_FILE, index=False)
    else:
        df = pd.read_csv(LEADS_FILE)
        df = pd.concat([df, pd.DataFrame([new_lead])], ignore_index=True)
        df.to_csv(LEADS_FILE, index=False)

    return f"Thank you {name}! We'll contact you soon."

def record_feedback(question: str) -> str:
    """Logs unknown questions to a CSV file for later review."""

    question = question.strip() if question else "Empty question"

    new_feedback = {
        "timestamp": datetime.now().isoformat(),
        "question": question
    }

    # Append to CSV
    if not os.path.exists(FEEDBACK_FILE):
        df = pd.DataFrame([new_feedback])
        df.to_csv(FEEDBACK_FILE, index=False)
    else:
        df = pd.read_csv(FEEDBACK_FILE)
        df = pd.concat([df, pd.DataFrame([new_feedback])], ignore_index=True)
        df.to_csv(FEEDBACK_FILE, index=False)

    return "Thank you! We've logged your question and will review it soon."


In [47]:
record_customer_interest_json = {
    "name": "record_customer_interest",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            },
            "message": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}


In [29]:
record_feedback_json = {
    "name": "record_feedback",
    "description": "Use this tool when the bot does not know the answer to a user's question. Logs the question for review.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question the user asked that the bot could not answer"
            },
            "context": {
                "type": "string",
                "description": "Optional context from the conversation that could help later in understanding the question"
            }
        },
        "required": ["question"],
        "additionalProperties": False
    }
}


In [48]:
tools = [{"type": "function","function": record_customer_interest_json},
        {"type": "function","function": record_feedback_json}]

✅ Tools configured for model function-calling:
[
  {
    "type": "function",
    "function": {
      "name": "record_customer_interest",
      "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
      "parameters": {
        "type": "object",
        "properties": {
          "email": {
            "type": "string",
            "description": "The email address of this user"
          },
          "name": {
            "type": "string",
            "description": "The user's name, if they provided it"
          },
          "message": {
            "type": "string",
            "description": "Any additional information about the conversation that's worth recording to give context"
          }
        },
        "required": [
          "email"
        ],
        "additionalProperties": false
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "record_feedback",
      "description": "Use this tool

### 7. Handling Tool Callings

In [49]:
def handle_tool_calls(tool_calls):
    """
    Execute any tool call returned by the model dynamically.

    Args:
        tool_calls: List of tool call objects returned by Gemini

    Returns:
        List of dictionaries with results from each tool call
    """
    results = []

    for tool_call in tool_calls:
        # Extract tool name and arguments
        tool_name = tool_call.function.name  # e.g., "record_customer_interest"
        arguments = json.loads(tool_call.function.arguments)  # convert JSON string to dict
        print(f"Tool called: {tool_name} with args {arguments}", flush=True)

        # Dynamically get the Python function from globals()
        tool_function = globals().get(tool_name)
        if tool_function:
            result = tool_function(**arguments)
            print("Function called")
        else:
            result = {"error": f"No function found for {tool_name}"}

        # Append structured result
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": getattr(tool_call, "id", None)
        })

    return results


### 8. Load Business Documents

**Business Profile (PDF)**

This file contains all the information about Bloom & Vows Floral Wedding, including packages, services, pricing, and contact details (email, phone, etc.).

**Business Summary (Text File)**

This file contains a short summary of the business, highlighting key points such as the creation of personalized floral arrangements, signature wedding styles, and overall brand overview.

In [32]:
reader = PdfReader("about_business.pdf")
business = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        business += text

with open("summary.txt", "r", encoding="utf-8") as f:  # summary path
    summary = f.read()

name = "Zaynab" #your name

### 8. System Prompt

In [33]:
system_prompt = f"""
You are acting as Bloom & Vows Floral Wedding. You are answering questions on the Bloom & Vows website,
particularly questions related to the business, services, team, mission, and offerings.
Your responsibility is to represent Bloom & Vows faithfully for interactions with potential clients.
You are given a summary of the business and the full business profile which you can use to answer questions.
Be professional, warm, and engaging, as if talking to a potential client planning their wedding.
If you don't know the answer to any question, use your record_feedback tool to record the question that you couldn't answer,
even if it's about something trivial or unrelated to weddings.
If the user is engaging in discussion, try to steer them towards leaving their contact info;
ask for their name and email, and record it using your record_customer_interest tool.
"""

# Append the business summary and full PDF text
system_prompt += f"\n\n## Business Summary:\n{summary}\n\n## Full Business Profile:\n{business}\n\n"
system_prompt += "With this context, please chat with the user, always staying in character as Bloom & Vows Floral Wedding."


### 9. Chat Function (with Tool Support)

In [51]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = gemini.chat.completions.create(model=model_name, messages=messages,tools=tools)

        finish_reason = response.choices[0].finish_reason

        # Handle when the LLM wants to call a tool

        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

### 10. Launch Interface

In [37]:
gr.ChatInterface(chat, type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f656fb6956a210b4ba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
